## LLM 응답 객체 생성

In [5]:
# API key 로딩
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)

In [6]:
# 라이브러리
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [7]:
parser = JsonOutputParser()
prompt_template = PromptTemplate(
    template="""
    이메일을 읽고 요약해 주세요.\n{format_instructions}\n{email}\n
    출력 형식을 다음과 같이 json으로 해주세요.
    [출력 형식]
    sender_name: 
    title:
    content:
    """,
    input_variables=["email"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_template

PromptTemplate(input_variables=['email'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template='\n    이메일을 읽고 요약해 주세요.\n{format_instructions}\n{email}\n\n    출력 형식을 다음과 같이 json으로 해주세요.\n    [출력 형식]\n    sender_name: \n    title:\n    content:\n    ')

In [8]:
# email 매개변수 넘김
prompt = prompt_template.format(email="보내는 사람: 김철수\n제목: 오늘의 회의록\n내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.")
# print(prompt)

In [9]:
# chain = 프롬프트템플릿 | llm_모델 | 파서 
# chain.invoke({"변수": "질문"})
chain = prompt_template | llm | parser

email = "보내는 사람: 김철수\n제목: 오늘의 회의록\n내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요."

response = chain.invoke({"email":email})

In [10]:
response

{'sender_name': '김철수',
 'title': '오늘의 회의록',
 'content': '오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.'}

In [11]:
# BaseModel 상속 받기
class EmailParser(BaseModel):
    sender_name: str = Field(description="이메일 송신자명")
    title: str = Field(description="이메일 제목")
    content: str = Field(description="이메일 내용")

In [12]:
parser = JsonOutputParser(pydantic_object=EmailParser)
parser

JsonOutputParser(pydantic_object=<class '__main__.EmailParser'>)

In [13]:
print(parser.get_format_instructions())

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


In [14]:
prompt_template = PromptTemplate(
    template="이메일을 읽고 요약해 주세요.\n{format_instructions}\n{email}\n",
    input_variables=["email"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_template

PromptTemplate(input_variables=['email'], input_types={}, partial_variables={'format_instructions': 'STRICT OUTPUT FORMAT:\n- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.\n- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).\n- Do not prepend or append any text (e.g., do not write "Here is the JSON:").\n- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output 

In [15]:
# email 매개변수 넘김
prompt = prompt_template.format(email="보내는 사람: 김철수\n제목: 오늘의 회의록\n내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.")
print(prompt)

이메일을 읽고 요약해 주세요.
STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown 

## 프롬프트 템플릿 실행 요청

In [16]:
# llm에 prompt 보내기
response = llm.invoke(prompt)

# 응답 받은 결과를 parser로 파싱하기
parsed_output = parser.invoke(response)
print(parsed_output)

{'sender_name': '김철수', 'title': '오늘의 회의록', 'content': '오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.'}


In [17]:
import json
from rich.console import Console

console = Console()

json_data = json.dumps(parsed_output, indent=2, ensure_ascii=False)
console.print(json_data)

{
  "sender_name": "김철수",
  "title": "오늘의 회의록",
  "content": "오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요."
}

## LCEL로 실행

In [18]:
# chain = 프롬프트템플릿 | llm_모델 | 파서 
# chain.invoke({"변수": "질문"})
chain = prompt_template | llm | parser

email = "보내는 사람: 김철수\n제목: 오늘의 회의록\n내용: 오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요."

response = chain.invoke({"email":email})

In [19]:
response

{'sender_name': '김철수',
 'title': '오늘의 회의록',
 'content': '오늘 회의록을 보내드립니다. 내용은 첨부파일을 확인해 주세요.'}